In [103]:
import os,sys,json
import kfp
from dkube.sdk import *

In [104]:
components_url = "/mnt/dkube/pipeline/components/"
dkube_training_op = kfp.components.load_component_from_file(components_url + "training/component.yaml")
dkube_serving_op = kfp.components.load_component_from_file(components_url + "serving/component.yaml")
token = os.getenv("DKUBE_USER_ACCESS_TOKEN")
client = kfp.Client(existing_token=token)

In [105]:
def get_dataset_version(data_versions):
    for i in range(len(data_versions)):
        if data_versions[i]['version']['name'] == 'pg_internal':
            return data_versions[i]['version']['uuid']

In [106]:
token = os.getenv("DKUBE_USER_ACCESS_TOKEN")
username =  os.getenv("USERNAME")
dkube_url = os.getenv("DKUBE_URL")
api = DkubeApi(URL=dkube_url,token=token)
data_versions = api.get_dataset_versions(username,"google-stock")
     
        
dataset_version = get_dataset_version(data_versions)

In [107]:
@kfp.dsl.pipeline(
    name='stock-redshift-pl',
    description='sample redshift pipeline with dkube components'
)
def stock_redshift_pipeline(token):

        train       = dkube_training_op(auth_token = token, container = '{"image":"docker.io/ocdr/d3-datascience-sklearn:v0.23.2"}',
                                    framework="sklearn", version="0.23.2",
                                    program="stock-prediction", run_script="python train.py",
                                    datasets='["google-stock"]', input_dataset_versions=json.dumps([dataset_version]),
                                    outputs='["stock-pred"]',
                                    input_dataset_mounts='["/opt/dkube/input"]',
                                    output_mounts='["/opt/dkube/model"]')
        
        eval_stage       = dkube_training_op(auth_token = token, container = '{"image":"docker.io/ocdr/d3-datascience-sklearn:v0.23.2"}',
                                    framework="sklearn", version="0.23.2",
                                    program="stock-prediction", run_script="python evaluation.py",
                                    datasets='["google-stock"]',input_dataset_versions=json.dumps([dataset_version]), 
                                    models='["stock-pred"]',
                                    input_dataset_mounts='["/opt/dkube/input"]',
                                    input_model_mounts='["/opt/dkube/model"]').after(train).set_display_name("evaluation")

        
        serving     = dkube_serving_op(auth_token = token, model= train.outputs['artifact'], device='cpu', 
                                    serving_image = '{"image": "ocdr/sklearnserver:0.23.2"}',
                                    transformer_image ='{"image":"docker.io/ocdr/d3-datascience-sklearn:v0.23.2"}',
                                    transformer_project = "stock-prediction",
                                    transformer_code = 'sklearn/stock-pred-redshift/model/transformer.py').after(train)

In [108]:
client.create_run_from_pipeline_func(stock_redshift_pipeline, arguments={"token":token})

RunPipelineResult(run_id=3f10f265-6b18-4130-9ce6-1d5b2f463e83)

In [109]:


#generate & upload pipeline (Optional)
import kfp.compiler as compiler
compiler.Compiler().compile(stock_redshift_pipeline, "stock_redshift_pipeline.zip")
client.upload_pipeline("stock_redshift_pipeline.zip")



{'created_at': datetime.datetime(2021, 5, 26, 11, 52, 42, tzinfo=tzlocal()),
 'default_version': {'code_source_url': None,
                     'created_at': datetime.datetime(2021, 5, 26, 11, 52, 42, tzinfo=tzlocal()),
                     'id': '5f2e5c70-3f49-45ff-9df0-5066e51d170a',
                     'name': 'stock_redshift_pipeline.zip',
                     'package_url': None,
                     'parameters': [{'name': 'token', 'value': None}],
                     'resource_references': [{'key': {'id': '5f2e5c70-3f49-45ff-9df0-5066e51d170a',
                                                      'type': 'PIPELINE'},
                                              'name': None,
                                              'relationship': 'OWNER'}]},
 'description': None,
 'error': None,
 'id': '5f2e5c70-3f49-45ff-9df0-5066e51d170a',
 'name': 'stock_redshift_pipeline.zip',
 'parameters': [{'name': 'token', 'value': None}],
 'resource_references': None,
 'url': None}